# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Choose the topic ✅

In [ ]:
import sys
sys.path.append("../src")

from test_chatbot import TestEris, TestSocrates
from chatbot import Socrates, Eris
from speak import stream_message, stream_first_message

import gradio as gr
import random

In [ ]:
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
# !winget install ffmpeg        # Required with pydub

## Debate: Claude VS GPT

### GUI

In [3]:
def adversarial_chat(n_rounds, topic, speed=0.01, stop_flag=None):
    """
    Enhanced debate with proper streaming and typing effects.
    """
    # socrates = TestSocrates()
    # eris = TestEris()
    socrates = Socrates()
    eris = Eris()
    history = []

    # Start the heated debate - use simple streaming for the opening message
    eris_options = [
        f"Ready to get schooled about '{topic}'?, old man?", 
        f"Today's topic is: '{topic}'. This should be easy—your logic is about as sharp as a wet napkin.",
        f"Hope you're prepared to lose spectacularly, Socrates. Let us talk about: '{topic}'",
        f"Hey Socrates. Wanna argue about: '{topic}'? It will be fun to see you cry once you have lost.",
    ]
    eris_reply = random.choice(eris_options)
    print("First", history)
    for update in stream_first_message(history, eris_reply, is_socrates=False, delay=speed):
        yield update

    # Continue with alternating turns using real API streaming
    for i in range(n_rounds-1):

        # -> History is automatically updated in the streaming function because it is passed by reference

        if i % 2 == 0:
            # Socrates takes even turns
            socrates.history = eris.history.copy()  # Sync conversation history
            for update in stream_message(history, socrates, eris_reply, is_socrates=True, delay=speed):
                yield update
            soc_reply = history[-1]["content"]
            
        else:
            #  Eris takes odd turns
            eris.history = socrates.history.copy()  # Sync conversation history
            for update in stream_message(history, eris, soc_reply, is_socrates=False, delay=speed):
                yield update
            eris_reply = history[-1]["content"]



In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0, minimum=1)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate", interactive=False)  # Initially disabled

    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=500)
    audio_output = gr.Audio(label="🗣️", autoplay=True, visible=True)

    def reset_chat():
        return [], None

    def validate_inputs(topic, n_rounds):
        valid_topic = bool(topic.strip())
        valid_rounds = isinstance(n_rounds, (int, float)) and n_rounds > 0
        return gr.update(interactive=valid_topic and valid_rounds)

    # Hook to clear previous chat/audio
    start_btn.click(fn=reset_chat, outputs=[chatbox, audio_output])

    # Launch the debate
    start_btn.click(fn=adversarial_chat,
                    inputs=[rounds, topic_input],
                    outputs=[chatbox, audio_output])

    # Enable/disable start button based on both inputs
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)


demo.launch(share=False)
# demo.launch(share=True, debug=True)
# demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


First []


d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


*adjusts toga and raises an eyebrow* Ah, Eris, your attempt to provoke me is as transparent as a child's plea for attention. If you wish to engage in intellectual discourse, I suggest you bring forth a substantive argument rather than mere provocative rhetoric.

Oh, Socrates, telling me to bring a "substantive argument" is rich coming from a font of philosophical clichés—surely you just enjoy repeating yourself endlessly, like a broken record. But then again, what can I expect from someone whose idea of wisdom is just echoing ancient platitudes?